# **DDL (Definition)** 🆔

In [2]:
import os
from pathlib import Path

import duckdb as dd
from faker import Faker
import pandas as pd

from config import CREDIT_RISK_DATA_DIR, DATABASE_DIR

2025-07-20 16:12:13.993 | INFO     | config:<module>:11 - PROJ_ROOT path is: C:\Users\mario\OneDrive\Documents\Education\guide_repo


## **DuckDB Connection**

In DuckDB, databases are either stored as files or kept `in memory`, which can be created like this. This command creates an in-memory database, meaning all operations are performed in the memory and won’t persist after the session ends.

```python
# Create an in-memory DuckDB connection
con = dd.connect(':memory:')
```

Alternatively, you can create a persistent DuckDB database by specifying a file path. This stores the database in the on-disk file `duckdb_test.db` in the current working directory, making it persistent between sessions.

In [4]:
# Create a persistent DuckDB database
os.chdir(DATABASE_DIR)
con = dd.connect("duckdb_test.db")

FILE_PATH = Path(CREDIT_RISK_DATA_DIR).as_posix()
INPUT_FILE = "credit_risk_dataset"

DuckDB supports standard SQL syntax, so you can run any SQL query with ease. Let’s start by creating an on-file/persistent database and querying it to see it’s contents.

Once you’re done with your queries, always remember to close the DuckDB connection.

In [ ]:
# con.close()

## **Creating Tables** 

The CREATE TABLE statement creates a new table in a database. It allows one to specify the name of the table and the name of each column in the table. In a CREATE TABLE statement we specify the data type for each column of a table (e.g., int, text, timestamp, etc.). Column constraints are the rules applied to the values of individual columns:

- PRIMARY KEY constraint can be used to uniquely identify the row.
- NOT NULL columns must have a value.
- CHECK columns must meet a pre-defined Boolean expression.
- UNIQUE columns have a different value for every row.
- DEFAULT assigns a default value for the column when no value is specified.

The SERIAL datatype creates an autoincrementing column that can be used as an Primary Key column.

In [6]:
con.execute(
    """
    DROP TABLE IF EXISTS storage;            
    DROP TABLE IF EXISTS users;            

    CREATE OR REPLACE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER CHECK (age > 0),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        email TEXT UNIQUE
    );
"""
)

In [7]:
con.execute(
    """
    DROP TABLE IF EXISTS reporting;  

    CREATE OR REPLACE TABLE reporting (
        person_age INT,
        person_income INT,
        person_home_ownership VARCHAR,
        person_emp_length FLOAT,
        loan_intent VARCHAR,
        loan_grade VARCHAR,
        loan_amnt INT,
        loan_int_rate FLOAT,
        loan_status INT,
        loan_percent_income FLOAT,
        cb_person_default_on_file VARCHAR,
        cb_person_cred_hist_length INT
    );
"""
)

## **Ingesting Tables**

We can ingest into tables from Pandas or Polars dataframes.

In [8]:
# Initialize the Faker object
fake = Faker()
sample = 100

# Create a dummy data set
dummy_data = {
    "name": [fake.name() for _ in range(sample)],
    "age": [fake.random_int(min=18, max=80) for _ in range(sample)],
    "created_at": [fake.date_time_this_decade() for _ in range(sample)],
}

# Convert the dummy data to a DataFrame
dummy_df = pd.DataFrame(dummy_data)
dummy_df["email"] = dummy_df.apply(
    lambda x: f"{x['name'].replace(' ', '.').lower()}@example.com", axis=1
)
dummy_df.reset_index(inplace=True, names="id")
dummy_df.head()

,id,name,age,created_at,email
0,0,John Vincent,52,2025-05-19 22:28:17,john.vincent@example.com
1,1,Jessica Carey,67,2021-08-29 08:22:24,jessica.carey@example.com
2,2,David Pratt,31,2024-06-24 20:48:34,david.pratt@example.com
3,3,Jennifer Allen,69,2024-10-12 15:37:12,jennifer.allen@example.com
4,4,Cory Baker,30,2021-04-10 08:32:59,cory.baker@example.com


In [9]:
query = """
    INSERT INTO users BY NAME 
    SELECT * FROM dummy_df
"""

con.execute(query)

In the examples above we saw, DuckDB gives us the capability to create tables and allows us to manually add data to them. However, if we are talking about large sets of data, we can ingest data from a variety of sources, including CSV, Parquet, JSON, etc. files. DuckDB lets us capture and store this data in a database.

In [11]:
query = f"""
    INSERT INTO reporting BY NAME 
    SELECT * FROM "{FILE_PATH}/{INPUT_FILE}.csv"
"""

con.execute(query)

In [12]:
con.sql("SELECT * FROM reporting LIMIT 5;")

┌────────────┬───────────────┬───────────────────────┬───────────────────┬─────────────┬────────────┬───────────┬───────────────┬─────────────┬─────────────────────┬───────────────────────────┬────────────────────────────┐
│ person_age │ person_income │ person_home_ownership │ person_emp_length │ loan_intent │ loan_grade │ loan_amnt │ loan_int_rate │ loan_status │ loan_percent_income │ cb_person_default_on_file │ cb_person_cred_hist_length │
│   int32    │     int32     │        varchar        │       float       │   varchar   │  varchar   │   int32   │     float     │    int32    │        float        │          varchar          │           int32            │
├────────────┼───────────────┼───────────────────────┼───────────────────┼─────────────┼────────────┼───────────┼───────────────┼─────────────┼─────────────────────┼───────────────────────────┼────────────────────────────┤
│         22 │         59000 │ RENT                  │             123.0 │ PERSONAL    │ D          │     35

## **Altering Tables**


#### **Add Columns**
The ALTER TABLE statement is used to modify the columns of an existing table. When combined with the ADD clause, it is used to add a new column. 

In SQLite, you can add multiple columns to an existing table using the ALTER TABLE statement. However, SQLite does not support adding multiple columns in a single ALTER TABLE statement. Instead, you must execute separate ALTER TABLE commands for each column you want to add.

In [ ]:
query = """
    ALTER TABLE users
    ADD COLUMN last_name TEXT;
    
    ALTER TABLE users
    ADD COLUMN test TEXT;
"""
con.execute(query)

con.sql("SELECT * FROM users LIMIT 5")

┌───────┬────────────────┬───────┬─────────────────────┬────────────────────────────┬───────────┬─────────┐
│  id   │      name      │  age  │     created_at      │           email            │ last_name │  test   │
│ int32 │    varchar     │ int32 │      timestamp      │          varchar           │  varchar  │ varchar │
├───────┼────────────────┼───────┼─────────────────────┼────────────────────────────┼───────────┼─────────┤
│     0 │ John Vincent   │    52 │ 2025-05-19 22:28:17 │ john.vincent@example.com   │ NULL      │ NULL    │
│     1 │ Jessica Carey  │    67 │ 2021-08-29 08:22:24 │ jessica.carey@example.com  │ NULL      │ NULL    │
│     2 │ David Pratt    │    31 │ 2024-06-24 20:48:34 │ david.pratt@example.com    │ NULL      │ NULL    │
│     3 │ Jennifer Allen │    69 │ 2024-10-12 15:37:12 │ jennifer.allen@example.com │ NULL      │ NULL    │
│     4 │ Cory Baker     │    30 │ 2021-04-10 08:32:59 │ cory.baker@example.com     │ NULL      │ NULL    │
└───────┴────────────────┴──

#### **Delete Columns**
We can also use the DROP COLUMN clause to remove a column.

In [17]:
query = """
    ALTER TABLE users
    DROP COLUMN test
"""
con.execute(query)

con.sql("SELECT * FROM users LIMIT 5")

┌───────┬────────────────┬───────┬─────────────────────┬────────────────────────────┬───────────┐
│  id   │      name      │  age  │     created_at      │           email            │ last_name │
│ int32 │    varchar     │ int32 │      timestamp      │          varchar           │  varchar  │
├───────┼────────────────┼───────┼─────────────────────┼────────────────────────────┼───────────┤
│     0 │ John Vincent   │    52 │ 2025-05-19 22:28:17 │ john.vincent@example.com   │ NULL      │
│     1 │ Jessica Carey  │    67 │ 2021-08-29 08:22:24 │ jessica.carey@example.com  │ NULL      │
│     2 │ David Pratt    │    31 │ 2024-06-24 20:48:34 │ david.pratt@example.com    │ NULL      │
│     3 │ Jennifer Allen │    69 │ 2024-10-12 15:37:12 │ jennifer.allen@example.com │ NULL      │
│     4 │ Cory Baker     │    30 │ 2021-04-10 08:32:59 │ cory.baker@example.com     │ NULL      │
└───────┴────────────────┴───────┴─────────────────────┴────────────────────────────┴───────────┘

#### **Identify Constraints**

Constraints are rules defined as part of the data model to control what values are allowed in specific columns and tables. Specifically, constraints reject inserts or updates containing values that shouldn’t be inserted into a database table, which can help with preserving data integrity and quality. They can also raise an error when they’re violated, which can help with debugging applications that write to the DB. 

Sometimes we’ve planned out a data model and inserted data before realizing that our model could benefit from the addition of a constraint. To find the constraints applied to a specific column, query the PostgreSQL catalog tables.

In [19]:
query = """
    SELECT conname, pg_get_constraintdef(c.oid) AS constraint_definition, a.attname AS column_name
    FROM pg_constraint c
    JOIN pg_class t ON c.conrelid = t.oid
    JOIN pg_attribute a ON a.attnum = ANY(c.conkey)
    WHERE t.relname = 'users';
    """

df = con.sql(query).df()
df[df.column_name == "id"]

,conname,constraint_definition,column_name
140,NOT NULL,NOT NULL,id


#### **Delete Constraints**


In [23]:
# query = """
#     ALTER TABLE users
#     DROP CONSTRAINT users_email_key;
# """

#### **Add Constraints**

Constraints are rules defined as part of the data model to control what values are allowed in specific columns and tables. Specifically, constraints reject inserts or updates containing values that shouldn’t be inserted into a database table, which can help with preserving data integrity and quality. They can also raise an error when they’re violated, which can help with debugging applications that write to the DB. 

Sometimes we’ve planned out a data model and inserted data before realizing that our model could benefit from the addition of a constraint. 

##### **Not Null**

In [21]:
query = """
    ALTER TABLE users
    ALTER COLUMN age SET NOT NULL;
"""

con.execute(query)

Deleting a NOT NULL constraint

In [22]:
query = """
    ALTER TABLE users
    ALTER COLUMN age DROP NOT NULL;
"""

con.execute(query)

##### **Check**

In some situations, we might want to establish specific rules to determine what makes a row valid. The condition tested for inside of parentheses of a CHECK statement must be a SQL statement that can be evaluated as either true or false. As a rule, any logic that you might use in a WHERE statement to filter individual rows from an existing table can be applied within a CHECK, including logic that involves multiple columns or conditions.

In [132]:
# query = """
#     ALTER TABLE users
#     ADD CHECK (age <= 100);
# """

# con.execute(query)

##### **Single-column Unique**

When designing a PostgreSQL data model, it’s a good practice to structure tables such that rows are uniquely identifiable by some combination of attributes. Identifying and implementing a PRIMARY KEY is easier on tables with UNIQUE constraints already in place. 

In [133]:
# query = """
#     ALTER TABLE users
#     ADD UNIQUE (email);
# """

# con.execute(query)

##### **Multi-column Unique**

We can create a multi-column UNIQUE constraint in the CREATE TABLE statement by specifying the columns that need to be jointly unique in parentheses on its own line following the column names and datatype definitions.

In [134]:
# query = """
#     ALTER TABLE users
#     ADD UNIQUE (email, phone);
# """

# con.execute(query)

##### **Primary Key**

Having unique constraints is useful, but an important part of building a relational data model requires defining relationships between tables. Primary keys are essential to defining these relationships. A primary key is a column (or set of columns) that uniquely identifies a row within a database table. A table can only have one primary key, and to be selected as a primary key a column (or set of columns) should:
- Uniquely identify that row in the table (like a UNIQUE constraint)
- Contain no null values (like a NOT NULL constraint)

Implementing a PRIMARY KEY constraint is similar to simultaneously enforcing a UNIQUE and NOT NULL constraints on a column (or set of columns). Although UNIQUE NOT NULL and PRIMARY KEY constraints function very similarly, tables are limited to one PRIMARY KEY, but not limited in how many columns can have both UNIQUE and NOT NULL constraints.


In [135]:
# query = """
#     ALTER TABLE users
#     ADD PRIMARY KEY (id);
# """

# con.execute(query)

##### **Composite Primary Key**

Sometimes, none of the columns in a table can uniquely identify a record. When this happens, we can designate multiple columns in a table to serve as the primary key, also known as a composite primary key.

In [ ]:
# query = """
#     ALTER TABLE users
#     ADD PRIMARY KEY (email, created_at);
# """

# con.execute(query)

##### **Foreign Key**

When discussing relations between tables, you may see the terms parent table and child table to describe two tables that are related. More specifically, values inserted into child table must be validated by data that’s already present in a parent table.
Formally, this property that ensures data can be validated by referencing another table in the data model is called referential integrity. Referential integrity can be enforced by adding a FOREIGN KEY on the child table that references the primary key of a parent table.

To designate a foreign key on a single column in PostgreSQL, we use the REFERENCES keyword:


In [28]:
query = """
    CREATE OR REPLACE TABLE storage (
    id INTEGER PRIMARY KEY,
    user_id INTEGER REFERENCES users(id),
    space INTEGER,
    price FLOAT
    );
"""
con.execute(query)

In [29]:
# Create a dummy data set for storage
storage_data = {
    "user_id": [fake.random_int(min=1, max=99) for _ in range(sample)],
    "space": [fake.random_int(min=1, max=1000) for _ in range(sample)],
    "price": [fake.random_number(digits=5) for _ in range(sample)],
}

# Convert the dummy data to a DataFrame
storage_df = pd.DataFrame(storage_data)
storage_df.reset_index(inplace=True, names="id")
storage_df.head()

,id,user_id,space,price
0,0,58,215,3088
1,1,68,32,57759
2,2,87,575,21899
3,3,23,856,86126
4,4,83,799,44973


In [30]:
query = """
    INSERT INTO storage BY NAME 
    SELECT * FROM storage_df
"""

con.execute(query)

Using an ALTER TABLE statement it would be this:

In [31]:
# query = """
#     ALTER TABLE storage
#     ADD FOREIGN KEY (user_id) REFERENCES users(id);
# """

# con.execute(query)

##### **Cascade Constraint**

By default, a foreign key constraint will prevent an engineer from deleting or updating a row of a parent table that is referenced by some child table. This behaviour is sometimes explicitly specified in a `CREATE TABLE` statement using `REFERENCES column_name(id) ON DELETE RESTRICT` or `REFERENCES column_name(id) ON UPDATE RESTRICT`. However, another strategy you may consider is adding a `CASCADE` clause.

In [ ]:
# query = """
#     ALTER TABLE storage
#     ADD FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE ON UPDATE CASCADE;
# """

# con.execute(query)

## **Indexing Tables**


An index is an organization of the data in a table to help with performance when searching and filtering records. A table can have zero, one, or many indexes. There are some costs when using indexes.

By default, it divides the possible matching records in half, then half, then half, and so on until the specific match you are searching for is found. This is known as a Binary Tree, or B-Tree. In small databases this is negligible, but as the datasets get larger this becomes more significant. To highlight this, let us say you were searching 1,000,000 records. Without an index on the column you were searching, you would need to look through all 1,000,000 records (assuming its a non-unique column). With a B-Tree index, in the worst case, you would have to search 20 comparisons (log2n).

Indexes speed up searching and filtering, however, they slow down insert, update, and delete statements. 

- When we add a record to a table that has an index, the index itself must be modified by the server as well. Recall that at its core, an index is an organization of the data in a table. When new data is added, the index will be reshaped to fit that new data into its organization. This means that when you write a single statement to modify the records, the server will have to modify every index that would be impacted by this change.
- Updates and deletes have similar drawbacks. When deleting a record that is associated with an index, it might be faster to find the record — by leveraging the index’s ability to search. However, once the record is found, removing or editing it will result in the same issue as inserting a new record. The index itself will need to be redone. Note that if you’re updating a non-indexed column, that update will be unaffected by the index. So if you are updating a non-indexed column while filtering by one with an index, an update statement can actually be faster with an index.


#### **Identify Indexes**

PostgreSQL automatically creates a unique on any primary key you have in your tables. It will also do this for any column you define as having a unique constraint. A unique index, primary key, and unique constraint all reject any attempt to have two records in a table that would have the same value (multicolumns versions of these would reject any record where all the columns are equal).
As a note, the primary key index standard is to end in _pkey instead of _idx to identify it as a specific type of index. It is also the way the system names it when created automatically.

##### **Existence**

This function returns which indexes exist in your table.


In [35]:
query = """
    SELECT *
    FROM pg_indexes
"""

df = con.sql(query).df()
df

,schemaname,tablename,indexname,tablespace,indexdef
0,main,users,users_age_idx,<NA>,CREATE INDEX users_age_idx ON users(age);


##### **Size**
The index data structures can sometimes take up as much space as the table itself. If you have not worked with large databases before you might be thinking to yourself, “who cares, storage space is cheap nowadays”. However, databases of decent size can easily get into the gigabyte size range quickly. If you wanted to examine the size of a table products you would run:


In [ ]:
# query = """
#     SELECT pg_size_pretty (pg_total_relation_size('users'));
# """
# df = con.sql(query).df()
# df

#### **Deleting Indexes**

If you are adding a large amount of data to an existing table, it may be better to drop the index, add the data, and then recreate the index rather than having to update the index on each insertion.


In [33]:
query = """
    DROP INDEX IF EXISTS users_age_idx;
"""
con.execute(query)

#### **Creating Indexes**

Don’t forget the naming convention for indexes, `<table_name>_<column_name>_idx`. 



##### **Single-column Indexes**

In [34]:
query = """
    CREATE INDEX users_age_idx ON users(age);
"""
con.execute(query)

##### **Multi-column Indexes**

Much like constraints, you can combine multiple columns together as a single index. When using multicolumn indexes, the search structure will be based on the values found in all the columns. For example, an index on First and Last Name might be a good idea if it is common to search by both together in your situation. Consider a table where the last names 'Smith' and 'Johnson' appear many times. Having another filter for the first name can help you find someone named 'Sarah Smith' much faster.

The index is built in the specific order listed at creation, so (last_name, first_name) is different from (first_name, last_name). Keep this in mind when you are building your indexes as the order will impact the efficiency of your searches.
Say you want to find 'David', 'Rachel', and 'Margaret' from the first_name column with the last_name of 'Smith'. If there is an index (last_name, first_name), the server would find everyone with the last name 'Smith' then in that much smaller group, find the specific first names you are searching for. If the index is (first_name, last_name) the server would go to each of the first_name records you are interested in and then search for the last name 'Smith' within each one. 

If there is a good use for it, you could create both indexes as well! If both are present, when you run your script, the database server will determine which index to use based on your query. 


In [36]:
# query = """
#     CREATE INDEX reporting_product_type_customer_segment_idx ON reporting(product_type, customer_segment);
# """
# con.execute(query)

##### **Partial Indexes**

A partial index allows for indexing on a subset of a table, allowing searches to be conducted on just this group of records in the table. All you have to do is create an index like you normally would with a WHERE clause added on to specify the subgroup of data your index should encompass. Let’s assume that in our example the users are stored in a users table and we want an index based on `name`. Note that the filtering of the index does not have to be for a column that is part of your index. 

If we know that all internal employees have an email_address ending in '@wellsfargo.com', we would write the partial index like this:


In [37]:
# query = """
#     CREATE INDEX users_name_internal_idx ON users(name)
#     WHERE email LIKE '%@wellsfargo.com';
# """
# con.execute(query)

##### **Ordered Indexes**

If you are commonly ordering your data in a specific way on an indexed column, you can add this information to the index itself and PostgreSQL will store the data in your desired order. By doing this, the results that are returned to you will already be sorted. You won’t need a second step of sorting them, saving time on your query.

You could also use `ASC` to switch the direction. If your column contains `NULL`s you can also specify the order they appear by adding `NULLS FIRST` or `NULLS LAST` to fit your needs. By default, PostgreSQL orders indexes by ascending order with `NULL`s last, so if this is the order you desire, you do not need to do anything.


In [38]:
query = """
    CREATE INDEX users_date_ordered_idx ON users(created_at DESC NULLS FIRST);
"""
con.execute(query)

##### **Expression-based Indexes**

An index is not limited to just a column reference, it can use the result of a function or scalar expression computed from one or more columns. 

In PostgreSQL, 'ExampleCompany' is NOT the same thing as 'examplecompany' even though we would probably want to reject this as a duplicate. You can add a function on your index to convert all your company_name data to lower case by using `LOWER`. This ensures that 'ExampleCompany' would be considered the same as 'examplecompany'. 

If you need flexibility, such as creating a unique index on a computed value or a subset of rows (e.g., partial indexes), create a unique index directly since creating a `UNIQUE` constraint on a table column directly does not allow this. A `UNIQUE` constraint applied to a column is a declarative constraint at the schema level, and PostgreSQL enforces it by automatically creating a unique index for that column.


In [39]:
query = """
    CREATE UNIQUE INDEX unique_users_name_idx ON users(LOWER(name));
"""
con.execute(query)

#### **Clustering Indexes**

A clustered index is often tied to the table’s primary key. When a clustered index is created for a table, the data is physically organized in the table structure to allow for improved search times. 

You can think of the clustered index like searching a dictionary. In a dictionary, the data (words) and all their related information (definition) are physically ordered by their index (words sorted alphabetically). Just like a dictionary, you can seek your word by quickly jumping to the letter in the alphabet the word you’re looking for starts with. Because it is physically organizing the data in the table, there can only be one clustered index per table.

When the system creates, alters, or refreshes a clustered index, it takes all the records in your database table that are in memory and rearranges them to match the order of your clustered index, physically altering their location in storage. Then when you go to do your searches for records based on this index, the system can use this index to find your records faster.

Something to note that PostgreSQL does differently than other systems is that it does not maintain this order automatically. When inserting data into a table with a clustered index on other systems, those systems will place the new records and altered records in their correct location in the database order in memory. PostgreSQL keeps modified records where they are and adds new records to the end, regardless of sorting. If you want to maintain the order, you must run the CLUSTER command again on the index when there have been changes. This will “re-cluster” the index to put all of those new records in the correct place.

Because PostgreSQL does not automatically recluster on `INSERT`, `UPDATE` and `DELETE` statements, those statements might run faster than equivalent statements using a different system. The flip side of this coin though is that after time, the more your table is modified the less useful the cluster will be on your searches. Reclustering the table has a cost, so you will need to find a balance on when to recluster your table(s). There are tools that can be used to help you identify when this would be useful, but these tools fall outside of this lesson.



To cluster your database table using an existing index (say products_product_name_idx) on the products table you would use:

In [180]:
# query = """
#     CLUSTER reporting USING reporting_product_type_customer_segment_idx;
# """
# con.execute(query)

If you have already established what index should be clustered on you can simply tell the system which table to apply the cluster on.

In [181]:
# query = """
#     CLUSTER reporting;
# """
# con.execute(query)

And if you want to cluster every table in your database that has an identified index to use you can simply call.

In [183]:
# query = """
#     CLUSTER;
# """
# con.execute(query)

##### **Non-clustered Indexes**

Previously we compared how a clustered index functions as a dictionary. You can think of all other indexes (non-clustered) more akin to an index in a book. The keywords you are looking for are organized (by type, alphabetically, by the number of appearances, etc) and can be found quickly. However, the index doesn’t contain information beyond that. Instead, it contains a pointer (page number, paragraph number, etc) to where the rest of the data can be found. This is the same way non-clustered indexes in databases work. You have a key that is sorted and a pointer to where to find the rest of the data if needed.

PostgreSQL defaults all indexes to non-clustered unless you specify one to be clustered. So you do not need to do anything special when creating your index for it to be non-clustered.

When you search on a non-clustered index for more information than is in the indexed columns, there are two searches. The first to find the record in the index and another to find the record the pointer identifies. There are some things you can do, such as creating a multicolumn index, that in some cases can help cut down or eliminate the need for the look back to the main table in memory.

##### **Expression-based Indexes**

An index is not limited to just a column reference, it can use the result of a function or scalar expression computed from one or more columns. 

In PostgreSQL, 'ExampleCompany' is NOT the same thing as 'examplecompany' even though we would probably want to reject this as a duplicate. You can add a function on your index to convert all your company_name data to lower case by using `LOWER`. This ensures that 'ExampleCompany' would be considered the same as 'examplecompany'. 

If you need flexibility, such as creating a unique index on a computed value or a subset of rows (e.g., partial indexes), create a unique index directly since creating a `UNIQUE` constraint on a table column directly does not allow this. A `UNIQUE` constraint applied to a column is a declarative constraint at the schema level, and PostgreSQL enforces it by automatically creating a unique index for that column.


In [40]:
query = """
    CREATE UNIQUE INDEX unique_users_email_idx ON users(LOWER(email));
"""
con.execute(query)

#### **Deciding on Indexes**

1. Stable tables: As a very rough rule of thumb, think carefully about any index on a table that gets regular Insert/Update/Delete.In contrast, a table that is fairly stable but is searched regularly might be a good candidate for an index.
2. Needle in a haystack: The higher the percentage of a table you are returning the less useful an index becomes. If we’re only searching for 1 record in 1,000,000, an index could be incredibly useful. However, if we are searching for 900,000 out of that same 1,000,000 the advantages of an index become useless. At higher percentages, the query planner might completely ignore your index and do a full table scan, making your index only a burden on the system.
3. AND is good, OR is bad: Along this same line, if you are combining filtering conditions be aware of what you will be searching on. AND statements are normally fine and the query planner will try to use an indexed field before non-indexed fields to cut down on the total number of records needed to be searched. OR on the other hand, can be very dangerous; even if you have a single non-indexed condition, if it’s in an OR, the system will still have to check every record in your table, making your index useless.
4. Single multi-column vs Multiple single columns

   - A multicolumn index is less efficient than a single index in cases where a single index is needed.
   - A single multicolumn index is faster (if ordered well) than the server combining single indexes.
   - You could create all of them (two single indexes and one multicolumn index), and then the server will try to use the best one in each case, but if they are all not used relatively often/equally then this is a misuse of indexes.

    Take for example, searching for first_name and last_name in the users table.

   - If searches are most often for only one of the columns, then you should use that single index. 
   - If searches are most often last_name and first_name, then you should have a multicolumn index. 
   - If the searches are frequent and evenly spread among first_name alone, last_name alone, and the combination of the two; that is a situation where you would want to have all three indexes (two single indexes and one multicolumn index).


#### **Analyse Performance**

To get insight into how PostgreSQL breaks down your statements into runnable parts, we can investigate the query plan by adding `EXPLAIN ANALYZE` before your query. Rather than returning the results of the query, it will return information about the query. For now, there are a few key things you should take note of. The first is the planner will specifically tell you how it is searching. If you see “Seq Scan” this means that the system is scanning every record to find the specific records you are looking for. If you see “Index” (in our examples more specifically “Bitmap Index Scan”) you know that the server is taking advantage of an index to improve the speed of your search.

The other part to take note of is the “Planning time” and “Execution time”. The planning time is the amount of time the server spends deciding the best way to solve your query, should it use an index, or do a full scan of the table(s) for instance. The execution time is the amount of time the actual query takes to run after the server has decided on a plan of attack. You need to take both of these into consideration, and when examining your own indexes these are critical to understanding how effective your indexes are.



In [41]:
query = """
    EXPLAIN ANALYZE 
    SELECT * FROM users
    WHERE age > 50;
"""
df = con.sql(query).df()

In [ ]:
# ┌─────────────────────────────────────┐
# │┌───────────────────────────────────┐│
# ││    Query Profiling Information    ││
# │└───────────────────────────────────┘│
# └─────────────────────────────────────┘
#      EXPLAIN ANALYZE      SELECT * FROM users     WHERE age > 50;
# ┌────────────────────────────────────────────────┐
# │┌──────────────────────────────────────────────┐│
# ││              Total Time: 0.0007s             ││
# │└──────────────────────────────────────────────┘│
# └────────────────────────────────────────────────┘
# ┌───────────────────────────┐
# │           QUERY           │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      EXPLAIN_ANALYZE      │
# │    ────────────────────   │
# │           0 Rows          │
# │          (0.00s)          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │         PROJECTION        │
# │    ────────────────────   │
# │             id            │
# │            name           │
# │            age            │
# │         created_at        │
# │           email           │
# │                           │
# │          46 Rows          │
# │          (0.00s)          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │         TABLE_SCAN        │
# │    ────────────────────   │
# │        Table: users       │
# │      Type: Index Scan     │
# │                           │
# │        Projections:       │
# │            age            │
# │             id            │
# │            name           │
# │         created_at        │
# │           email           │
# │                           │
# │      Filters: age>50      │
# │                           │
# │          46 Rows          │
# │          (0.00s)          │
# └───────────────────────────┘